<a href="https://colab.research.google.com/github/avdlaan/Angular-10-Templates/blob/master/Answer_Extractor_functions_from_Data_set_with_Ktrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip3 install ktrain==0.28.2
!pip install tensorflow==2.4.1

In [ ]:
import numpy as np
import pandas as pd 
import glob
import json
import math

Set the initial imports- for the next step let us iterate the directory.

In [ ]:
all_json = glob.glob('/content/drive/MyDrive/train/document_parses/json_test/*.json', recursive=True)

# let us get the list of articles
len(all_json)
print(len(all_json))

39


In [ ]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            # Abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:400]}... {self.body_text[:500]}...'

In [ ]:
first_row = FileReader(all_json[0])
print(first_row)

000a0fc8bbef80410199e690191dc3076a290117: Background: Various transcription factors are involved in the process of mutually exclusive expression and clonal variation of the Plasmodium multigene (var) family. Recent studies revealed that a P. falciparum SWI/SNF-related matrix-associated actin-dependent regulator of chromatin (PfSWIB) might trigger stage-specific programmed cell death (PCD), and was not only crucial for the survival and dev... Plasmodium falciparum, which causes malignant malaria such as cerebral malaria (CM) or pregnancy-associated malaria (PAM) [1, 2] , is the cause of death to 435,000 individuals annually [3] . Plasmodium falciparum erythrocyte membrane protein 1 (PfEMP1), which is encoded by the var gene family comprising approximately 60 members, is the major virulence factor involved in the antigenic variation and clinical pathogenicity of falciparum malaria [4, 5] . It is remarkable that the expression of var f...


In [ ]:
dict_ = {'paper_id': [], 'abstract': [], 'body_text': []}
for idx, entry in enumerate(all_json):
    try:
        content = FileReader(entry)
        if (idx % 1000 == 0):
            print(f"processing {idx} of {len(all_json)} ")
        dict_['paper_id'].append(content.paper_id)
        dict_['abstract'].append(content.abstract)
        dict_['body_text'].append(content.body_text)
    except:
        pass
df_covid = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text'])
df_covid.head(39)

processing 0 of 39 


,paper_id,abstract,body_text
0,000a0fc8bbef80410199e690191dc3076a290117,Background: Various transcription factors are ...,"Plasmodium falciparum, which causes malignant ..."
1,00a0df94d685622b8a7894f6a952e97a7e89ccf6,"Citation: Kostyuk, S.V.; Proskurnina, E.V.; Er...",Recovery of muscle tissue damaged by diseases ...
2,00a00d0edc750db4a0c299dd1ec0c6871f5a4f24,,This is an Open Access article distributed und...
3,00a1a921b93d9773f46d21ac22b1363371c7d535,Efforts to mitigate the global spread of the s...,a1111111111 a1111111111 a1111111111 a111111111...
4,00a0ab182dc01b6c2e737dfae585f050dcf9a7a5,,The world was made aware of a newly discovered...
5,00a0c02539f89317beb2954f72b20fa91dd0cc87,,"On 23 March 2020, the UK announced the ""stay a..."
6,0a0c7465fb4d6dfc7c5c3b4d019bf12a9a3cea6b,The spread of coronavirus disease infection ac...,The global pandemic caused by the coronavirus ...
7,0a1a319ffd87d5b69e7738e57c382a3fc3c8b420,,"In December 2019, the severe acute respiratory..."
8,0a00fe902ce5f523ceca3d01dc68dd1ddaac253b,We propose the Modified Corona Score (MCorona ...,"On behalf of all authors, the corresponding au..."
9,0a0d9d7f7ccd750820cbe07415db17e4d8ae2f2d,The emergence of a novel strain of betacoronav...,To respond effectively to the recent Severe Ac...


In [ ]:
import ktrain
from ktrain.text.qa import AnswerExtractor

ae = AnswerExtractor()

df_covid = ae.extract(df_covid.body_text.values, df_covid, [('What are the proteins in Covid Virus?', 'Proteins'),
                                     ('Effects of Covid?','Effects'),('What the Covid Virus Strains?','Strains')])

df_covid.head(39)

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

,paper_id,abstract,body_text,Proteins,Effects,Strains
0,000a0fc8bbef80410199e690191dc3076a290117,Background: Various transcription factors are ...,"Plasmodium falciparum, which causes malignant ...",None,None,None
1,00a0df94d685622b8a7894f6a952e97a7e89ccf6,"Citation: Kostyuk, S.V.; Proskurnina, E.V.; Er...",Recovery of muscle tissue damaged by diseases ...,None,None,None
2,00a00d0edc750db4a0c299dd1ec0c6871f5a4f24,,This is an Open Access article distributed und...,None,None,None
3,00a1a921b93d9773f46d21ac22b1363371c7d535,Efforts to mitigate the global spread of the s...,a1111111111 a1111111111 a1111111111 a111111111...,None,increased public fear,None
4,00a0ab182dc01b6c2e737dfae585f050dcf9a7a5,,The world was made aware of a newly discovered...,None,None,None
5,00a0c02539f89317beb2954f72b20fa91dd0cc87,,"On 23 March 2020, the UK announced the ""stay a...",None,None,None
6,0a0c7465fb4d6dfc7c5c3b4d019bf12a9a3cea6b,The spread of coronavirus disease infection ac...,The global pandemic caused by the coronavirus ...,None,The global pandemic caused by the coronavirus ...,None
7,0a1a319ffd87d5b69e7738e57c382a3fc3c8b420,,"In December 2019, the severe acute respiratory...",None,None,None
8,0a00fe902ce5f523ceca3d01dc68dd1ddaac253b,We propose the Modified Corona Score (MCorona ...,"On behalf of all authors, the corresponding au...",None,Covid 19 causes a clinical syndrome encompassi...,None
9,0a0d9d7f7ccd750820cbe07415db17e4d8ae2f2d,The emergence of a novel strain of betacoronav...,To respond effectively to the recent Severe Ac...,None,None,None


In [ ]:
#df_covid = ae.extract(df_covid.body_text.values, df_covid, [('Covid Recovery?', 'Recovery'),
#                                     ('Long Covid signs?','Signs')])
#df_covid.head(12)